![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **6. Context Spell Checker - Medical**


In [1]:
import json
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from johnsnowlabs import nlp, medical

spark = start_spark()
spark.sparkContext.setLogLevel("ERROR")

spark

📋 Loading license number 0 from /home/ubuntu/.johnsnowlabs/licenses/license_number_0_for_.json


23/01/06 13:57:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, 🕶Spark-OCR==4.2.4, running on ⚡ PySpark==3.1.2


In [3]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.RecursiveTokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    .setPrefixes(["\"", "(", "[", "\n"])\
    .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = nlp.ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 134.7 MB
[ | ]spellcheck_clinical download started this may take some time.
Approximate size to download 134.7 MB
[ | ]Download done! Loading the resource.


[ / ]

[ \ ]

[OK!]


In [4]:
pipeline = nlp.Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
    ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = nlp.LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,

_She was **treathed** with a five day course of **amoxicilin** for a **resperatory** **truct** infection._

_With pain well controlled on **orall** **meditation**, she was discharged to **reihabilitation** **facilitay**._


_Her **adominal** examination is soft, nontender, and **nonintended**_

_The patient was seen by the **entocrinology** service and she was discharged on 40 units of **unsilin** glargine at night_
      
_No __cute__ distress_

Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [5]:
example = ["She was treathed with a five day course of amoxicilin for a resperatory truct infection . ",
           "With pain well controlled on orall meditation, she was discharged to reihabilitation facilitay.",
           "Her adominal examination is soft, nontender, and nonintended.",
           "The patient was seen by the entocrinology service and she was discharged on 40 units of unsilin glargine at night",
           "No cute distress",
          ]

for pairs in lp.annotate(example):
    print(list(zip(pairs['token'],pairs['checked'])))

[('She', 'She'), ('was', 'was'), ('treathed', 'treated'), ('with', 'with'), ('a', 'a'), ('five', 'five'), ('day', 'day'), ('course', 'course'), ('of', 'of'), ('amoxicilin', 'amoxicillin'), ('for', 'for'), ('a', 'a'), ('resperatory', 'respiratory'), ('truct', 'tract'), ('infection', 'infection'), ('.', '.')]
[('With', 'With'), ('pain', 'pain'), ('well', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('meditation', 'medication'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('to', 'to'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('Her', 'Her'), ('adominal', 'abdominal'), ('examination', 'examination'), ('is', 'is'), ('soft', 'soft'), (',', ','), ('nontender', 'nontender'), (',', ','), ('and', 'and'), ('nonintended', 'nondistended'), ('.', '.')]
[('The', 'The'), ('patient', 'patient'), ('was', 'was'), ('seen', 'seen'), ('by', 'by'), ('the', 'the'), ('entocrinology', 'endocrinology'), ('service', 'se

In [6]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['checked'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:



[('treathed', 'treated'),
 ('amoxicilin', 'amoxicillin'),
 ('resperatory', 'respiratory'),
 ('truct', 'tract'),
 ('orall', 'oral'),
 ('meditation', 'medication'),
 ('reihabilitation', 'rehabilitation'),
 ('facilitay', 'facility'),
 ('adominal', 'abdominal'),
 ('nonintended', 'nondistended'),
 ('entocrinology', 'endocrinology'),
 ('unsilin', 'insulin'),
 ('cute', 'acute')]